In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("input"))

# Any results you write to the current directory are saved as output.

['.DS_Store', 'test.csv', 'train.csv']


In [84]:
import pandas as pd
import string, re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.metrics import confusion_matrix
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import WordNetLemmatizer

In [157]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [126]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [12]:
train_raw_df = pd.read_csv("input/train.csv")
test_df = pd.read_csv("input/test.csv")
train_n = len(train_df)
test_n = len(test_df)
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (130612, 3)
Test shape :  (56370, 2)


In [135]:
train_df = train_raw_df.sample(frac=0.05)
print("Train shape : ",train_df.shape)
train_df.head(1)

Train shape :  (65306, 3)


,qid,question_text,target
737723,907a93d2f27d74b9a224,When did London lose its manners?,1


In [136]:
lemmatizer = WordNetLemmatizer()
stopset = set(stopwords.words('english'))

In [137]:
def process(txt):
    tokens = []
    for token in wordpunct_tokenize(txt):
        if token.isdigit():
            continue
        if all(char in string.punctuation for char in token):
            continue
        
        token = token.lower()
        token = token.strip()  # Strip whitespace and other punctuations
        token = token.strip('_')  # remove _ if any
        token = token.strip('*')
        if token in stopset:
            continue
        tokens.append(token)
        lemmatizer.lemmatize(token)
    return tokens

In [155]:
train_df['p_txt'] = train_df['question_text'].apply(process)
test_df['p_txt'] = test_df['question_text'].apply(process)

In [139]:
train_df.head(1)

,qid,question_text,target,p_txt
737723,907a93d2f27d74b9a224,When did London lose its manners?,1,"[london, lose, manners]"


In [156]:
train_df['p_txtcn'] = train_df['p_txt'].apply(lambda tokens: ' '.join(str(v) for v in tokens))
test_df['p_txtcn'] = test_df['p_txt'].apply(lambda tokens: ' '.join(str(v) for v in tokens))

In [141]:
no_features = 1000
tfidf_vectorizer = TfidfVectorizer(max_features=no_features)

In [142]:
def vectorize(txt, vectorizer):
    X = vectorizer.fit_transform(txt)
    feature_names = vectorizer.get_feature_names()
    return X, feature_names

In [143]:
X_tf, tf_feature_names = vectorize(train_df['p_txtcn'], tfidf_vectorizer)

In [144]:
no_topics = 10
num_iter = 2

In [145]:
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=num_iter, learning_method='online', learning_offset=50.,random_state=9, evaluate_every=100).fit(X_tf)

In [146]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        top_features = []
        for i in topic.argsort()[:-no_top_words - 1:-1]:
            if topic[i] >  1.0:
                top_features.append((feature_names[i], topic[i]))
            else:
                continue        
        print("Topic %d:" % (topic_idx))
        print(", ".join([str(val[0])+": "+"%.2f" % val[1] for val in top_features]))

In [147]:
display_topics(lda, tf_feature_names, 50)

Topic 0:
best: 927.17, make: 557.96, life: 536.23, way: 480.90, better: 378.16, country: 287.58, would: 287.41, year: 269.22, long: 249.43, old: 248.40, say: 245.31, day: 215.46, live: 212.35, new: 201.18, years: 198.91, girl: 195.31, known: 143.50, eat: 140.83, friend: 138.39, another: 129.41, win: 127.12, found: 125.32, market: 115.66, believe: 110.35, less: 110.25, weight: 106.89, fight: 99.21, girlfriend: 94.76, die: 93.87, culture: 93.14, project: 91.62, term: 90.76, oil: 88.63, people: 88.29, modi: 84.46, talk: 83.82, skin: 82.17, case: 80.55, prevent: 78.88, modern: 78.05, rate: 76.65, exist: 76.15, control: 76.01, run: 75.63, sleep: 75.05, good: 74.83, price: 74.57, eating: 73.95, vs: 73.26, explain: 72.98
Topic 1:
become: 446.13, work: 432.94, take: 272.38, engineering: 246.62, anyone: 245.40, college: 241.32, without: 226.44, go: 224.12, learn: 203.51, getting: 193.47, get: 187.60, university: 185.89, look: 183.53, company: 179.51, like: 175.06, career: 170.12, students: 165.

In [148]:
X_lda = lda.transform(X_tf)

In [149]:
# X_lda[:30]

In [151]:
X_lda_df = pd.DataFrame(X_lda)
X_lda_df["idxmax"] = X_lda_df.idxmax(axis=1)
X_lda_df.head(2)

,0,1,2,3,4,5,6,7,8,9,idxmax
0,0.050000,0.050000,0.050000,0.050000,0.050000,0.550000,0.050000,0.050000,0.050000,0.050000,5
1,0.029061,0.503562,0.029061,0.029061,0.029061,0.029061,0.133664,0.029061,0.029061,0.159347,1


In [152]:
y_true = train_df['target']
y_lda = X_lda_df["idxmax"]

In [153]:
print(confusion_matrix(y_true, y_lda)[:2])

[[9865 6852 5490 5461 6203 4329 4459 6899 5707 6023]
 [ 478  316  434  265  314  365  525  238  321  762]]


In [161]:
embed_size = 100
max_features = None
maxlen = 100

tokenizer = Tokenizer(num_words=max_features, filters='')
tokenizer.fit_on_texts(train_df['p_txt'])

X = tokenizer.texts_to_sequences(train_df['p_txt'])
X_test = tokenizer.texts_to_sequences(test_df['p_txt'])
X = pad_sequences(X, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [162]:
X_test[:1]

array([[    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,   953,  1293, 11340, 23640,
         2029,   953,  1065,  2792,  3525,   709,  2029,  1293,   953,
          274]], dtype=int32)

In [164]:
word_index = tokenizer.word_index
max_features = len(word_index)+1